In [11]:
!pip install --upgrade openai gradio pillow requests

**ChatGPT API - Code working**

In [17]:
#!pip install --upgrade openai gradio pillow requests

import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import openai

# -----------------------------
# Set your OpenAI API key here
# -----------------------------

openai.api_key = "changedapikeysk-proj-apic4Q8vsmRmbEEf4HofpZ-Vv_7o1GF1e4ESHvlF7fFuaWGTWJJjXXDMd7DR65iVgwa1jLVcUpVtbT3BlbkFJxq2DXrixjDkUtAap_b2Vbz0AdabMM2WEhbcLslyVIh9jcamEoQLXsKhkoldDVJNS_2xGs07UQA"  # <-- replace with your real key


# -----------------------------
# Global chat history
# -----------------------------
chat_history = []



In [18]:
#!pip install --upgrade openai gradio pillow requests

import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import openai

# -----------------------------
# OpenAI API Key
# -----------------------------


# -----------------------------
# Global chat history
# -----------------------------
chat_history = []

# -----------------------------
# Preview image from URL
# -----------------------------
def preview_image(image_url):
    try:
        img_data = requests.get(image_url).content
        pil_img = Image.open(BytesIO(img_data))
        return pil_img
    except Exception as e:
        print(f"[DEBUG] Could not load image: {e}")
        return None

# -----------------------------
# Ask GPT-4o about image
# -----------------------------
def ask_medical_image(image_url, user_input):
    global chat_history

    if not user_input:
        return "", []

    # Append user message
    chat_history.append({"role": "user", "content": user_input})

    # Construct messages for OpenAI API
    messages = chat_history.copy()
    if image_url:
        messages.append({
            "role": "user",
            "content": f"[Image URL: {image_url}] Please consider this image while answering."
        })

    # Call OpenAI Chat API (>=1.0)
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=500,
        temperature=0.7
    )

    assistant_msg = response.choices[0].message.content

    # Append assistant response
    chat_history.append({"role": "assistant", "content": assistant_msg})

    # Format for Gradio Chatbot (list of dicts with role/content)
    return assistant_msg, chat_history

# -----------------------------
# Summarize conversation
# -----------------------------
def summarize_conversation():
    global chat_history
    if not chat_history:
        return "No conversation to summarize."

    summary_prompt = "Summarize the following conversation medically and concisely:\n"
    for msg in chat_history:
        summary_prompt += f"{msg['role']}: {msg['content']}\n"

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": summary_prompt}],
        max_tokens=300,
        temperature=0.7
    )

    return response.choices[0].message.content

# -----------------------------
# Diagnostic / treatment suggestions
# -----------------------------
def diagnostic_suggestions():
    global chat_history
    if not chat_history:
        return "No conversation for diagnostics."

    diag_prompt = "Based on the following conversation, provide possible diagnostic or treatment suggestions:\n"
    for msg in chat_history:
        diag_prompt += f"{msg['role']}: {msg['content']}\n"

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": diag_prompt}],
        max_tokens=400,
        temperature=0.7
    )

    return response.choices[0].message.content

# -----------------------------
# Clear chat
# -----------------------------
def clear_chat():
    global chat_history
    chat_history = []
    return "", None, [], "", ""


In [19]:

# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Medical Assistant with ChatGPT (GPT-4o): Image + Chat + Summary + Diagnostics")

    with gr.Row():
        img_url_input = gr.Textbox(label="Image URL", placeholder="Paste any public medical image URL")
        preview_img = gr.Image(label="Image Preview")
    img_url_input.change(fn=preview_image, inputs=img_url_input, outputs=preview_img)

    question_input = gr.Textbox(label="Ask a question about the image", placeholder="Example: What abnormalities are present in this X-ray?")

    with gr.Row():
        submit_btn = gr.Button("Ask")
        clear_btn = gr.Button("Clear Chat")
        summary_btn = gr.Button("Summarize Conversation")
        diag_btn = gr.Button("Diagnostic / Treatment Suggestions")

    chat_output = gr.Chatbot(label="Conversation")
    answer_output = gr.Textbox(label=None, lines=10, placeholder="GPT-4o answer will appear here...")

    summary_output = gr.Textbox(label="Summary", lines=8, placeholder="Summary will appear here...")
    diag_output = gr.Textbox(label="Diagnostics / Treatment Suggestions", lines=12, placeholder="Diagnostics will appear here...")

    # Button actions
    submit_btn.click(
        fn=ask_medical_image,
        inputs=[img_url_input, question_input],
        outputs=[answer_output, chat_output]
    )

    clear_btn.click(
        fn=clear_chat,
        inputs=None,
        outputs=[answer_output, preview_img, chat_output, summary_output, diag_output]
    )

    summary_btn.click(
        fn=summarize_conversation,
        inputs=None,
        outputs=summary_output
    )

    diag_btn.click(
        fn=diagnostic_suggestions,
        inputs=None,
        outputs=diag_output
    )

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5c7b030ebf1fecfedf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://5c7b030ebf1fecfedf.gradio.live
